# Assignment 4
In the in-class workbook, we had a classifier which was designed to picks between two specific digits, one that we called **signal** and the other we called **background**.   

In this assignment, we will want to read in all of the digits, and design a classifier which finds a specific digit (our **signal** again), but **all** of the other 9 digits will serve as the background.   Our background will naturally be 9 times bigger than our signal (unless we limit it).



# A useful method
The "nested_defaultdict" function is very useful for making two dimensional counter (actually nested dictionaries).  


In [21]:
from collections import defaultdict
from functools import partial
from itertools import repeat
def nested_defaultdict(default_factory, depth=1):
    result = partial(defaultdict, default_factory)
    for _ in repeat(None, depth - 1):
        result = partial(defaultdict, result)
    return result()  


## Helpful for good plotly-express behavior

In [22]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default='notebook'

# Getting the data:
The data is available in the project space.   If you uncomment the line with **short = "short_"**, the code will run faster.  

**Once it is working you should use the full sample.**

In [23]:
import pandas as pd
#
short = ""
# short = "short_"

#
# Read in all of the other digits
dfAll = pd.DataFrame()
for digit in range(10):
    print("Processing digit ",digit)
    fname = '/fs/ess/PAS1759/data/ch3/digit_' + short + str(digit) + '.csv'
    df = pd.read_csv(fname,header=None)
    df['digit'] = digit
    dfAll = pd.concat([dfAll, df])
print(dfAll)

Processing digit  0
Processing digit  1
Processing digit  2
Processing digit  3
Processing digit  4
Processing digit  5
Processing digit  6
Processing digit  7
Processing digit  8
Processing digit  9
      0  1  2  3  4  5  6  7  8  9  ...  775  776  777  778  779  780  781  \
0     0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
1     0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
2     0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
3     0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
4     0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
...  .. .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
6953  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
6954  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
6955  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
6956  0  0  0  0  0 

# Task 0: Defining Signal
At the top of this block we define which of the 10 digits we want to use for our **signal**.  The default is 5.

Use dfA to contain just samples from this digit, and dfB to use samples from the other digits.   You should be able to pull these out of the dfAll dataframe, each with one line of code.  Hint: use .loc and .copy().

In [24]:
#
# Define our "signal" digit
digitSignal = 5

dfA = dfAll.loc[dfAll['digit'] == digitSignal].copy()
dfB = dfAll.loc[dfAll['digit'] != digitSignal].copy()

dfA['signal'] = 1     # dfA has the digit we will look for = the signal
dfB['signal'] = 0     # dfB has all of the other digits = the background

print("Length of signal sample:     ",len(dfA))
print("Length of background sample: ",len(dfB))


Length of signal sample:      6313
Length of background sample:  63687


# Task1: Shuffle background
You will need to **shuffle** or randomize the rows of the **background** dataframe.  We already read the digits in, but they were in batches of the the same digit.   So we need to shuffle them to mix the digits up.  To figure out how to do this, google **sklearn shuffle pandas**.   Note that the "shuffle" method from sklearn creates copies of the original dataframe (which is fine).

Put the shuffled dataframe in dfBShuffle.

In [25]:
from sklearn.utils import shuffle
dfBShuffle = shuffle(dfB, random_state=0)
print(dfBShuffle)

      0  1  2  3  4  5  6  7  8  9  ...  776  777  778  779  780  781  782  \
7372  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
6968  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
2666  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
5168  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
2880  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
...  .. .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
3280  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
2512  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
2     0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
956   0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
2732  0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   

      783  digit  signal  
7372    0      1       0  
6968    0

# Tasks 2: Limit background
Come up with a method to limit the rows of the background data that you use, so that it is the **same length** (in rows) as the signal dataframe.   You will want some easy way to turn this on and off.   Run first with the background limited to the **same** length as the signal.   Later you can come back and use all of the background data.

Call the limited dataframe dfB_use.


In [26]:
# Your code goes here

dfB_use = dfBShuffle.head(6313)

# Task 3: Combining data
After steps 1 and 2 , you will have a signal dataframe, and a randomized background dataframe.   You will need to combine these two into a single dataframe.  We have done this before, but you can look at **pandas concat** function.   Use the name **dfCombined** for the combined dataframe.


In [27]:
# Your code goes here

dfCombined = pd.concat([dfA, dfB_use], ignore_index=True)

print("Size of signal sample ",len(dfA))
print("Size of background sample ",len(dfB_use))
print("Size of combined sample ",len(dfCombined))



Size of signal sample  6313
Size of background sample  6313
Size of combined sample  12626


#  Task 4: runFitter Method
Next you will want to apply an estimator to this dataset. I want you to make a function that does both the test/train split, and then calls the estimator, just like we did at the end of the in-class workbook. The function should look like the following "skeleton".   I show the expected inputs and the expected return values.   Note we did all of this in the example workbook.

In [28]:
#
# Here is the skeleton of the method
from sklearn.model_selection import train_test_split
#
# The inputs are:
#     dfCombined: the input dataframe
#     estimator:  this should be an sklearn classifier (only LinearSVC or SGDClassifier are expected to be used)
def runFitter(dfCombined,estimator):
    #
    # First do a test/train split
    train_digits,test_digits = train_test_split(dfCombined, test_size=0.2, random_state=42)

    X_train = train_digits.iloc[:,:784].to_numpy()
    y_train = train_digits['signal'].to_numpy()
    y_train_digit = train_digits['digit'].to_numpy()

    X_test = test_digits.iloc[:,:784].to_numpy()
    y_test = test_digits['signal'].to_numpy()
    y_test_digit = test_digits['digit'].to_numpy()

    #
    # Now fit to our training set
    estimator.fit(X_train,y_train)
    #
    # Now predict the classes and get the score for our traing set
    y_train_pred = estimator.predict(X_train)
    y_train_score = estimator.decision_function(X_train)
    #
    # Now predict the classes and get the score for our test set
    y_test_pred = estimator.predict(X_test)
    y_test_score = estimator.decision_function(X_test)  
    #
    return y_train,y_train_pred,y_train_score,y_train_digit,y_test,y_test_pred,y_test_score,y_test_digit



# Task 5: Run the fitter
Now we can use the function we defined above.   We have to define our estimate (which we get from sklearn) outside of the method, and pass it as an argument to our defined function.   We do it this way because later on we will want to call the method with a different estimator

In [29]:
# Your code goes here
from sklearn.svm import LinearSVC

estimator = LinearSVC(random_state=42,dual=False,max_iter=5000)    # use dual=False when  n_samples > n_features which is what we have

# call runFitter
y_train,y_train_pred,y_train_score,y_train_digit,y_test,y_test_pred,y_test_score,y_test_digit = runFitter(dfCombined,estimator)

# Task 5: Implement Performance Method
Next, you need to get the performance of the estimator on **both** the training and testing data.   To do this, I want you to make a function to calulate various performance metrics, and return the result.   

In [30]:
#
# Determine the performance
#
# The inputs:
#    y = array of true labels
#    y_pred = array of predicted labels from the **predict** method of the LinearSVC estimator
#    y_score = array of scores from the **decision_function** method of the LinearSVC estimator
#
# The return values:
#    precision,recall,auc are the calculated values of these metrics
#    fpr, tpr, thresholds are lists containing the  "false positive rate", "true positve rate", and "threshold"
#
def binaryPerformance(y,y_pred,y_score,debug):
#
# Assuming a binary classifier with 1=signal, 0=background
    confusionMatrix = nested_defaultdict(int,2)
    for i in range(len(y_pred)):
        trueClass = y[i]      # this is either 0 or 1
        predClass = y_pred[i]
        confusionMatrix[trueClass][predClass] += 1
    
    data = [ [ confusionMatrix[1][1],confusionMatrix[1][0] ],
             [ confusionMatrix[0][1],confusionMatrix[0][0] ] ]
    df = pd.DataFrame(data)
    df.rename(columns={0:"Pred=DigitA", 1:"Pred=DigitB"},index={0:'True=DigitA',1:'True=DigitB'},inplace=True)
    
    TP = confusionMatrix[1][1]
    FP = confusionMatrix[0][1]
    FN = confusionMatrix[1][0]
    TN = confusionMatrix[0][0]
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)

    result = []
    data_type = []
    for i in range(len(y_pred)):
        result.append(y_score[i])
        if y[i]==1:
            data_type.append("Signal")
        else:
            data_type.append("Background")

    
    totalSignal = 0
    totalBackground = 0

    for dtype in data_type:
        if dtype=='Signal':
            totalSignal += 1
        else:
            totalBackground += 1

    #
    # Start the loop over thresholds
    maxScore = max(result)
    minScore = min(result)
    stepSize = (maxScore - minScore) / 100.0
    threshold = minScore
    fpr = []
    tpr = []
    thresholds = []

    for steps in range(100):
    #
    # At each step, count the number of TP and FP
        tn = 0.0
        tp = 0.0
        for score,dtype in zip(result,data_type):
            if dtype=='Signal' and score > threshold:
                tp += 1.0
            elif dtype=='Background' and score <= threshold:
                tn += 1.0

        #
        # Remember that "false negatives" or fn is the number of positives that were NOT tp
        fn = totalSignal - tp
        fp = totalBackground - tn
        #
        # Now get the rates for the curve
        tpRate = tp / totalSignal
        fpRate = fp / totalBackground

        fpr.append(fpRate)
        tpr.append(tpRate)
        thresholds.append("Threshold="+str(threshold))

        threshold += stepSize

    from sklearn.metrics import auc
    auc = auc(fpr,tpr)
    
    return precision,recall,auc,fpr, tpr, thresholds

# Task 6: Call performance method
Call the "binaryPerformance" method for both the training and testing results for your estimator. How do they compare? Print out the precision, recall, and AUC of the test and train samples.  The plot the ROC for both test and train.   You can do these plots in separate figures.

In [31]:
#
# Now get the performaance
precision_test,recall_test,auc_test,fpr_test, tpr_test, thresholds_test = binaryPerformance(y_test,y_test_pred,y_test_score,debug=False)
precision_train,recall_train,auc_train,fpr_train, tpr_train, thresholds_train = binaryPerformance(y_train,y_train_pred,y_train_score,debug=False)

print("AUC training data: ",auc_train)
print("AUC testing data:  ",auc_test)


AUC training data:  0.9845044592893302
AUC testing data:   0.9636037077042879


# Plot the ROC!

In [33]:
fig = px.area(x=fpr_test,y=tpr_test,labels={'x':'FPR', 'y':'TPR'},
                          hover_data={'Score':thresholds_test},title='ROC test Curve')
fig.show()

fig2 = px.area(x=fpr_train,y=tpr_train,labels={'x':'FPR', 'y':'TPR'},
                          hover_data={'Score':thresholds_train},title='ROC train Curve')
fig2.show()

# Task 7: A different Classifier
Run with a different classifier (and the large background statistics): the SGDClassifier. 

Using the "runFitter method and the binaryPerformance method should make this very easy.

As before, print out the precision, recall, and AUC of the test and train samples. The plot the ROC for both test and train. 

Compare your results to the LinearSVC classifier.

In [34]:
# Your code goes here
# run the fitter
from sklearn.linear_model import SGDClassifier

estimatorSGD = SGDClassifier(random_state=42) ##
# Test the fitter
y_train,y_train_pred,y_train_score,y_train_digit,y_test,y_test_pred,y_test_score,y_test_digit = runFitter(dfCombined,estimatorSGD)
# Now get the performaance
precision_test,recall_test,auc_sgd_test,fpr_test, tpr_test, thresholds_test = binaryPerformance(y_test,y_test_pred,y_test_score,debug=False)
precision_train,recall_train,auc_sgd_train,fpr_train, tpr_train, thresholds_train = binaryPerformance(y_train,y_train_pred,y_train_score,debug=False)

print("AUC training data: ",auc_train)
print("AUC testing data:  ",auc_test)
print("AUC SGD training data: ",auc_sgd_train)
print("AUC SGD testing data:  ",auc_sgd_test)


AUC training data:  0.9845044592893302
AUC testing data:   0.9636037077042879
AUC SGD training data:  0.9800924842579466
AUC SGD testing data:   0.9658447210429801


In [35]:
fig = px.area(x=fpr_test,y=tpr_test,labels={'x':'FPR', 'y':'TPR'},
                          hover_data={'Score':thresholds_test},title='ROC test Curve')
fig.show()

fig2 = px.area(x=fpr_train,y=tpr_train,labels={'x':'FPR', 'y':'TPR'},
                          hover_data={'Score':thresholds_train},title='ROC train Curve')
fig2.show()

In [38]:
#The ROC figrues using SGD were much smoother. It looks like SGD performs better on the test data than SVC.

# Important!!! Did you remember to rerun using the FULL data?

# Extra credit: Additional tasks:
1.   For a given signal digit, is the accuracy with which background is rejected, dependent upon what the background digit is?   For example, imagine that our signal digit is 5.   Is the accuracy that a 1 is identifed as background the same as the accuracy that a 8 is identified as background?  Hint: 8 is much less likely to be identified as background than 1 is.   The reasdon should be clear if you compare the shape of a 1 and an 8 to a 5.
2.   Limit the signal to 1/10 of its maximum (about 600), and the background to the same number (about 600).   What is the performance of the estimator?   Use LinearSVC for this.

## Background study

In [40]:
#
# short = ""
short = "short_"

#
# Read in all of the other digits
dfAll = pd.DataFrame()
for digit in range(10):
    fname = '/fs/ess/PAS1759/data/ch3/digit_' + short + str(digit) + '.csv'
    df = pd.read_csv(fname,header=None)
    df['digit'] = digit
    dfAll = pd.concat([dfAll, df])

#
# Define our "signal" digit
# and the background digit
digitSignal = 5
digitBackground = 8

dfA = dfAll.loc[dfAll['digit'] == digitSignal].copy()
dfB = dfAll.loc[dfAll['digit'] == digitBackground].copy()

dfA['signal'] = 1     # dfA has the digit we will look for = the signal
dfB['signal'] = 0     # dfB has all of the other digits = the background

dfCombined = pd.concat([dfA, dfB], ignore_index=True)

# Your code goes here
estimator = LinearSVC(random_state=42,dual=False,max_iter=5000)    # use dual=False when  n_samples > n_features which is what we have

# call runFitter
y_train,y_train_pred,y_train_score,y_train_digit,y_test,y_test_pred,y_test_score,y_test_digit = runFitter(dfCombined,estimator)

precision_test,recall_test,auc_test,fpr_test, tpr_test, thresholds_test = binaryPerformance(y_test,y_test_pred,y_test_score,debug=False)
precision_train,recall_train,auc_train,fpr_train, tpr_train, thresholds_train = binaryPerformance(y_train,y_train_pred,y_train_score,debug=False)

print("AUC training data: ",auc_train)
print("AUC testing data:  ",auc_test)

fig = px.area(x=fpr_test,y=tpr_test,labels={'x':'FPR', 'y':'TPR'},
                          hover_data={'Score':thresholds_test},title='ROC test Curve')
fig.show()

fig2 = px.area(x=fpr_train,y=tpr_train,labels={'x':'FPR', 'y':'TPR'},
                          hover_data={'Score':thresholds_train},title='ROC train Curve')
fig2.show()

##################################################################################################################

AUC training data:  0.9987484355444305
AUC testing data:   0.9760869021725542


In [41]:
#
# Define our "signal" digit
# and the background digit
digitSignal = 5
digitBackground = 1

dfA = dfAll.loc[dfAll['digit'] == digitSignal].copy()
dfB = dfAll.loc[dfAll['digit'] == digitBackground].copy()

dfA['signal'] = 1     # dfA has the digit we will look for = the signal
dfB['signal'] = 0     # dfB has all of the other digits = the background

dfCombined = pd.concat([dfA, dfB], ignore_index=True)

# Your code goes here
estimator = LinearSVC(random_state=42,dual=False,max_iter=5000)    # use dual=False when  n_samples > n_features which is what we have

# call runFitter
y_train,y_train_pred,y_train_score,y_train_digit,y_test,y_test_pred,y_test_score,y_test_digit = runFitter(dfCombined,estimator)

precision_test,recall_test,auc_test,fpr_test, tpr_test, thresholds_test = binaryPerformance(y_test,y_test_pred,y_test_score,debug=False)
precision_train,recall_train,auc_train,fpr_train, tpr_train, thresholds_train = binaryPerformance(y_train,y_train_pred,y_train_score,debug=False)

print("AUC training data: ",auc_train)
print("AUC testing data:  ",auc_test)

fig = px.area(x=fpr_test,y=tpr_test,labels={'x':'FPR', 'y':'TPR'},
                          hover_data={'Score':thresholds_test},title='ROC test Curve')
fig.show()

fig2 = px.area(x=fpr_train,y=tpr_train,labels={'x':'FPR', 'y':'TPR'},
                          hover_data={'Score':thresholds_train},title='ROC train Curve')
fig2.show()

AUC training data:  0.9987484355444305
AUC testing data:   0.9947623690592264


In [ ]:
# Performance is clearly better for 1 as the background digit

## Impact of less data

In [53]:
dfA = dfAll.loc[dfAll['digit'] == digitSignal].copy()
dfB = dfAll.loc[dfAll['digit'] != digitSignal].copy()

dfA['signal'] = 1     # dfA has the digit we will look for = the signal
dfB['signal'] = 0     # dfB has all of the other digits = the background

dfCombined = pd.concat([dfA.head(600), dfB_use.head(600)],ignore_index=True)

# Your code goes here
estimator = LinearSVC(random_state=42,dual=False,max_iter=5000)    # use dual=False when  n_samples > n_features which is what we have

# call runFitter
y_train,y_train_pred,y_train_score,y_train_digit,y_test,y_test_pred,y_test_score,y_test_digit = runFitter(dfCombined,estimator)

precision_test,recall_test,auc_test,fpr_test, tpr_test, thresholds_test = binaryPerformance(y_test,y_test_pred,y_test_score,debug=False)
precision_train,recall_train,auc_train,fpr_train, tpr_train, thresholds_train = binaryPerformance(y_train,y_train_pred,y_train_score,debug=False)

print("AUC training data: ",auc_train)
print("AUC testing data:  ",auc_test)

fig = px.area(x=fpr_test,y=tpr_test,labels={'x':'FPR', 'y':'TPR'},
                          hover_data={'Score':thresholds_test},title='ROC test Curve')
fig.show()

fig2 = px.area(x=fpr_train,y=tpr_train,labels={'x':'FPR', 'y':'TPR'},
                          hover_data={'Score':thresholds_train},title='ROC train Curve')
fig2.show()

AUC training data:  0.9979253112033195
AUC testing data:   0.9367532647957767
